<a href="https://colab.research.google.com/github/3adel/auto-correct/blob/main/training/t5_text_spelling_local_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#pip install transformers

In [22]:
#Setup logging

#from google.colab import drive
import datetime
import logging

#drive.mount('/content/drive/')

# Function to write logs with timestamp
now_log_file_name_suffix = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
log_file_path = f'/Users/adel/adel/dev/playground/auto-correct/logs/{now_log_file_name_suffix}-t5-text_spelling_trainer.log'
def log_message(message):
    # Get the current date and time
    now = datetime.datetime.now()
    # Format the current date and time as a string
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    # Prepare the log entry
    log_entry = f"{timestamp} - {message}\n"
    # Open the log file and append the log entry
    with open(log_file_path, 'a') as file:
        file.write(log_entry)

# Example usage
log_message("Execution starts")

In [23]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import random

# Load the pre-trained T5-small model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small", legacy=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
# Define the task-specific head
task_prefix = "correct: "


In [25]:
import pandas as pd

# URL pointing directly to the raw CSV data
url = 'https://raw.githubusercontent.com/3adel/auto-correct/main/training_sets_dumb/frequency-alpha-alldicts_1000_words.csv'

try:
    # Attempt to read the CSV file from the provided URL
    data = pd.read_csv(url)
    print("Data loaded successfully!")
except Exception as e:
    # Print the error if the file cannot be read
    print(f"An error occurred: {e}")

training_data = list(zip(data['misspelling'], data['correct_word']))


Data loaded successfully!


In [26]:
# Now text_pairs contains tuples in the format (incorrect, correct)
print("# rows in training data touples = ",len(training_data))
log_message(f"# rows in training data = {len(training_data)}")
log_message(f"############ Random seen data to be validated by the model later.")

seen_validation_data = []
number_seen_data_points = 100

for i in range (number_seen_data_points):
  random_range_start_index = random.randrange(1,9050,1)
  for row in training_data[random_range_start_index:random_range_start_index+1]:
    #print(row[0])
    seen_validation_data.append(row)
print(f"Seen data: {seen_validation_data}")
log_message(f"Seen data: {seen_validation_data}")


# rows in training data touples =  10223
Seen data: [('meetign', 'meeting'), (' boedi', 'body'), (' beguinn', 'began'), (' simlar', 'similar'), (' linnes', 'lines'), (' proposd', 'proposed'), (' sensas', 'sense'), (' furthurst', 'further'), ('tel', 'tell'), (' attetion', 'attention'), (' sael', 'sale'), (' constrcution', 'construction'), ('mye', 'my'), (' larig', 'large'), (' cettee', 'set'), (' saund', 'sound'), (' lates', 'late'), (' humaniin', 'human'), (' madee', 'made'), (' hadas', 'had'), (' writen', 'written'), (' reange', 'range'), (' suych', 'such'), (' othre', 'other'), (' otuo', 'out'), (' Wersten', 'Western'), (' plaingtiff', 'plaintiff'), (' othere', 'other'), (' posst', 'post'), (' paa', 'pay'), (' impotantce', 'importance'), (' firie', 'fire'), (' leftey', 'left'), (' twyords', 'toward'), (' suloi', 'soil'), ('ma', 'am'), (' Amerciia', 'America'), (' harid', 'hard'), (' momnet', 'moment'), (' prrioi', 'prior'), (' wiid', 'wide'), ('werre', 'were'), (' throguhout', 'throu

In [27]:
#Load unseen data by the model
unseen_validation_data = [
    ("Appel", "Apple"),
    ("Booke", "Book"),
    ("Kare", "Car"),
    ("Dög", "Dog"),
    ("Hoise", "House"),
    ("Trei", "Tree"),
    ("Freind", "Friend"),
    ("Hapy", "Happy"),
    ("Bieg", "Big"),
    ("Littel", "Little"),
    ("Rune", "Run"),
    ("Jummp", "Jump"),
    ("Sée", "See"),
    ("Heer", "Hear"),
    ("Saiy", "Say"),
    ("Goe", "Go"),
    ("Comey", "Come"),
    ("Stoip", "Stop"),
    ("Starit", "Start"),
    ("Fynish", "Finish"),
    ("Wont", "Want"),
    ("Neid", "Need"),
    ("Likie", "Like"),
    ("Lovve", "Love"),
    ("Heelp", "Help"),
    ("Tayk", "Take"),
    ("Givv", "Give"),
    ("Mekke", "Make"),
    ("Doe", "Do"),
    ("Noow", "Know"),
    ("Thinck", "Think"),
    ("Fiel", "Feel"),
    ("Sée", "See"),
    ("Heer", "Hear"),
    ("Seee", "See"),
    ("Heer", "Hear"),
    ("Eit", "Eat"),
    ("Drinik", "Drink"),
    ("Sleeep", "Sleep"),
    ("Wakie", "Wake"),
    ("Wurk", "Work"),
    ("Plaiy", "Play"),
    ("Lernn", "Learn"),
    ("Readd", "Read"),
    ("Rrite", "Write"),
    ("Tawk", "Talk"),
    ("Lissten", "Listen"),
    ("Smilie", "Smile"),
    ("Laught", "Laugh"),
    ("Crie", "Cry"),
    ("Yelle", "Yell"),
    ("Hurrie", "Hurry"),
    ("Waite", "Wait"),
    ("Hoipe", "Hope"),
    ("Wisshe", "Wish"),
    ("Trii", "Try"),
    ("Fale", "Fail"),
    ("Suceede", "Succeed"),
    ("Begn", "Begin"),
    ("Ennd", "End"),
    ("Mornig", "Morning"),
    ("Afteernoon", "Afternoon"),
    ("Eveninng", "Evening"),
    ("Nite", "Night"),
    ("Dei", "Day"),
    ("Wieck", "Week"),
    ("Montn", "Month"),
    ("Yeer", "Year"),
    ("Centurie", "Century"),
    ("Lief", "Life"),
    ("Tymme", "Time"),
    ("Spase", "Space"),
    ("Energee", "Energy"),
    ("Watair", "Water"),
    ("Firre", "Fire"),
    ("Aire", "Air"),
    ("Erth", "Earth"),
    ("Skye", "Sky"),
    ("Sone", "Sun"),
    ("Munne", "Moon"),
    ("Phenomonal", "Phenomenal"),
    ("Compitition", "Competition"),
    ("Independacently", "Independently"),
    ("Unrelevent", "Irrelevant"),
    ("Disapointment", "Disappointment"),
    ("Requirments", "Requirements"),
    ("Impossiblity", "Impossibility"),
    ("Exsceptional", "Exceptional"),
    ("Incurrable", "Incurred"),
    ("Unnoticable", "Unnoticeable"),
    ("Misunderstnding", "Misunderstanding"),
    ("Reccomendation", "Recommendation"),
    ("Incompatiblity", "Incompatibility"),
    ("Exsistential", "Existential"),
    ("Irrelevent", "Irrelevant"),
    ("Unpredicatable", "Unpredictable"),
    ("Disastorous", "Disastrous"),
    ("Accomplishement", "Accomplishment"),
    ("Inefficiency", "Inefficacy"),
    ("Impracticality", "Impracticability")
]

log_message(f"############ Unseen data to be validated by the model later.")
print(f"Number of rows in unseen validation data = {len(unseen_validation_data)}")
log_message(f"Number of rows in unseen validation data = {len(unseen_validation_data)}")


print(f"Unseen validation data: {unseen_validation_data}")
log_message(f"Unawwn validation data: {unseen_validation_data}")



Number of rows in unseen validation data = 100
Unseen validation data: [('Appel', 'Apple'), ('Booke', 'Book'), ('Kare', 'Car'), ('Dög', 'Dog'), ('Hoise', 'House'), ('Trei', 'Tree'), ('Freind', 'Friend'), ('Hapy', 'Happy'), ('Bieg', 'Big'), ('Littel', 'Little'), ('Rune', 'Run'), ('Jummp', 'Jump'), ('Sée', 'See'), ('Heer', 'Hear'), ('Saiy', 'Say'), ('Goe', 'Go'), ('Comey', 'Come'), ('Stoip', 'Stop'), ('Starit', 'Start'), ('Fynish', 'Finish'), ('Wont', 'Want'), ('Neid', 'Need'), ('Likie', 'Like'), ('Lovve', 'Love'), ('Heelp', 'Help'), ('Tayk', 'Take'), ('Givv', 'Give'), ('Mekke', 'Make'), ('Doe', 'Do'), ('Noow', 'Know'), ('Thinck', 'Think'), ('Fiel', 'Feel'), ('Sée', 'See'), ('Heer', 'Hear'), ('Seee', 'See'), ('Heer', 'Hear'), ('Eit', 'Eat'), ('Drinik', 'Drink'), ('Sleeep', 'Sleep'), ('Wakie', 'Wake'), ('Wurk', 'Work'), ('Plaiy', 'Play'), ('Lernn', 'Learn'), ('Readd', 'Read'), ('Rrite', 'Write'), ('Tawk', 'Talk'), ('Lissten', 'Listen'), ('Smilie', 'Smile'), ('Laught', 'Laugh'), ('Crie', '

In [28]:
# Preprocess the data
def preprocess_data(input_texts, target_texts):

    #To address the issue of potential non-string types and None values in your input_texts and target_texts
    inputs = [task_prefix + str(text) for text in input_texts if text is not None]
    target_texts = [str(text) for text in target_texts if text is not None]

    # Tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(target_texts, max_length=512, truncation=True, padding="max_length", return_tensors="pt")["input_ids"]
    return model_inputs, labels

# Prepare the training data
input_texts, target_texts = zip(*training_data)
train_inputs, train_labels = preprocess_data(input_texts, target_texts)

In [29]:
# Inference method
def correct_text(input_text):
    input_ids = tokenizer.encode(task_prefix + input_text, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=512)[0]
    corrected_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    return corrected_text

In [30]:
#run inference BEFORE finetuning against SEEN data
%%time
log_message(f"############ run inference BEFORE finetuning against SEEN data")
for i, row in enumerate(seen_validation_data):
  correct_text(row[0])
  print(f"{i}:{row[0]}--->{correct_text(row[0])}")
  log_message(f"{i}:{row[0]}--->{correct_text(row[0])}")



0:meetign--->meetign
1: boedi--->correct: boedi
2: beguinn--->: beguinn
3: simlar--->: simlar
4: linnes--->-
5: proposd--->correct: proposition
6: sensas--->correct: sensas
7: furthurst--->: furthurst
8:tel--->correct: tel
9: attetion--->correct: attetion
10: sael--->-
11: constrcution--->: constrcution
12:mye--->: mye
13: larig--->correct: larig
14: cettee--->correct: cettee
15: saund--->correct: saund
16: lates--->Richtig: lates
17: humaniin--->: humaniin
18: madee--->correct: madee
19: hadas--->- -
20: writen--->correct: writen
21: reange--->reange
22: suych--->suych
23: othre--->othre
24: otuo--->correct: otuo
25: Wersten--->Wersten: Wersten: Wersten: Wersten: Wersten: Wersten Wersten: Wersten: Wersten: Wersten
26: plaingtiff--->: plaingtiff
27: othere--->correct: correct: othere
28: posst--->Richtig: Posst
29: paa--->correct: paa
30: impotantce--->correct: impotantce
31: firie--->: firie
32: leftey--->leftey
33: twyords--->twyords
34: suloi--->suloi
35:ma--->correct: ma
36: Amerci

In [31]:
#run inference BEFORE finetuning against UNSEEN data
%%time
log_message(f"############ run inference BEFORE finetuning against UNSEEN data")
for i, row in enumerate(unseen_validation_data):
  correct_text(row[0])
  print(f"{i}:{row[0]}--->{correct_text(row[0])}")
  log_message(f"{i}:{row[0]}--->{correct_text(row[0])}")

0:Appel--->Appel
1:Booke--->Booke
2:Kare--->Kare
3:Dög--->Dög: Dög Dög: Dög Dög: Dög Dög: Dög: Dög: Dög: Dög: Dög: Dög
4:Hoise--->Hoise
5:Trei--->correct: Trei
6:Freind--->Freind: Freind
7:Hapy--->Hapy: Hapy
8:Bieg--->Bieg
9:Littel--->Littel Littel Littel: Littel Littel Littel Littel Littel: Littel
10:Rune--->Rune
11:Jummp--->Jummp: Jummp Jummp: Jummp Jummp Jummp Jummp: Jummp: Jummp: Jummp Jummp: Jummp
12:Sée--->Sé
13:Heer--->Heer: Heer: Heer: Heer Heer: Heer
14:Saiy--->Saiy
15:Goe--->Goe: Goe: Goe: Goe Goe: Goe Goe: Goe: Goe: Goe: Goe: Goe: Goe
16:Comey--->Comey
17:Stoip--->Stoip
18:Starit--->Starit: Starit
19:Fynish--->Fynish: Fynish: Fynish: Fynish Fynish: Fynish
20:Wont--->Wont
21:Neid--->correct: Neid
22:Likie--->Likie
23:Lovve--->: Lovve
24:Heelp--->Heelp
25:Tayk--->Tayk
26:Givv--->Givv
27:Mekke--->Mekke
28:Doe--->Doe
29:Noow--->Noow
30:Thinck--->: Thinck
31:Fiel--->Fiel
32:Sée--->Sé
33:Heer--->Heer: Heer: Heer: Heer Heer: Heer
34:Seee--->Seee
35:Heer--->Heer: Heer: Heer: Heer He

In [32]:
#fine tuning w/ batch procerssing
from torch.utils.data import DataLoader, TensorDataset

# Create a TensorDataset
dataset = TensorDataset(train_inputs["input_ids"], train_inputs["attention_mask"], train_labels)

# Create a DataLoader
batch_size = 16  # You can adjust the batch size according to your GPU memory
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print("# training rows per batch:", len(train_dataloader))
log_message(f"# training rows per batch = {len(train_dataloader)}")



# training rows per batch: 639


In [ ]:
# Fine-tune the model w/ frequent output
#%%time
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
num_epochs = 2

for epoch in range(num_epochs):
    total_loss = 0
    current_batch = 0

    for batch in train_dataloader:
        batch_start_time = datetime.datetime.now()
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Started batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs}")
        log_message(f"Started batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs}")

        # Unpack the batch
        batch_input_ids, batch_attention_mask, batch_labels = batch


        # Zero the gradients on each iteration
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)


        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Print progress
        batch_end_time = datetime.datetime.now()
        print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Finished batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs} in ({(batch_end_time - batch_start_time).total_seconds():.2f} seconds)")
        log_message(f"Finished batch #{current_batch+1}/{len(train_dataloader)} in epoch #{epoch+1}/{num_epochs}  in ({(batch_end_time - batch_start_time).total_seconds():.2f} seconds)")

        current_batch += 1



    # Calculate average loss over the number of batches
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
    log_message(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

2024-04-30 13:33:43 Started batch #1/639 in epoch #1/2
2024-04-30 13:33:53 Finished batch #1/639 in epoch #1/2 in (9.56 seconds)
2024-04-30 13:33:53 Started batch #2/639 in epoch #1/2
2024-04-30 13:34:02 Finished batch #2/639 in epoch #1/2 in (9.41 seconds)
2024-04-30 13:34:02 Started batch #3/639 in epoch #1/2
2024-04-30 13:34:11 Finished batch #3/639 in epoch #1/2 in (9.35 seconds)
2024-04-30 13:34:11 Started batch #4/639 in epoch #1/2
2024-04-30 13:34:21 Finished batch #4/639 in epoch #1/2 in (9.36 seconds)
2024-04-30 13:34:21 Started batch #5/639 in epoch #1/2
2024-04-30 13:34:30 Finished batch #5/639 in epoch #1/2 in (9.38 seconds)
2024-04-30 13:34:30 Started batch #6/639 in epoch #1/2
2024-04-30 13:34:39 Finished batch #6/639 in epoch #1/2 in (9.36 seconds)
2024-04-30 13:34:39 Started batch #7/639 in epoch #1/2
2024-04-30 13:34:49 Finished batch #7/639 in epoch #1/2 in (9.34 seconds)
2024-04-30 13:34:49 Started batch #8/639 in epoch #1/2
2024-04-30 13:34:58 Finished batch #8/639 

In [ ]:
#run inference AFTER finetuning against SEEN data
%%time
log_message(f"############ run inference AFTER finetuning against SEEN data")
for i, row in enumerate(seen_validation_data):
  correct_text(row[0])
  print(f"{i}:{row[0]}--->{correct_text(row[0])}")
  log_message(f"{i}:{row[0]}--->{correct_text(row[0])}")

In [ ]:
#run inference AFTER finetuning against UNSEEN data
%%time
log_message(f"############ run inference AFTER finetuning against UNSEEN data")
for i, row in enumerate(unseen_validation_data):
  correct_text(row[0])
  print(f"{i}:{row[0]}--->{correct_text(row[0])}")
  log_message(f"{i}:{row[0]}--->{correct_text(row[0])}")

In [ ]:
# Save the model and tokenizer
#Save model to Local Drive
finetuning_iteration = "7"
model.save_pretrained(f"/Users/adel/adel/dev/playground/auto-correct/models/{finetuning_iteration}")
tokenizer.save_pretrained(f"/Users/adel/adel/dev/playground/auto-correct/models/{finetuning_iteration}")


In [ ]:
# How to load the model from drive

finetuned_model_path = f"/Users/adel/adel/dev/playground/auto-correct/models/{finetuning_iteration}"
finetuned_tokenizer = T5Tokenizer.from_pretrained(finetuned_model_path)
model_finetuned_spelling_en_1000_words = T5ForConditionalGeneration.from_pretrained(finetuned_model_path)


In [ ]:
#use the finetuned model.
def correct_spelling(word):
    input_ids = tokenizer.encode("spelling correction:" + word, return_tensors="pt")

    # Correct spelling
    outputs = model_finetuned_spelling_en_1000_words.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Decode the output
    correction = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return correction

# Example text
word = "objeccts"
correction = correct_spelling(word)
print("Correction:", correction)
